<a href="https://colab.research.google.com/github/Ogeanni/webscrape-article/blob/main/HashnodeArticles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import pandas as pd
from collections import defaultdict
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer # to reduce words to orginal form
from nltk.corpus import words           # Get all words in english
# from emot.emo_unicode import UNICODE_EMOJI # For emojis
import string
import textblob
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
lemmatizer = WordNetLemmatizer()


In [2]:
def hashnode():
    url = 'https://hashnode.com/community?source=header'
    html_content = requests.get(url)
    soup = BeautifulSoup(html_content.text, 'lxml')
    container = soup.find('div', attrs={'data-state':'active'})
    return container


# def hashnode():
#     url = 'https://hashnode.com/community?source=header'

#     retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
#     session = requests.Session()
#     adapter = HTTPAdapter(max_retries=retries)
#     session.mount('https://', adapter)

#     try:
#         html_content = requests.get(url, timeout=10)
#         html_content.raise_for_status()
#     except requests.exceptions.RequestException as e:
#         print(f"Error during HTTP request: {e}")
#         return None  # Return None or handle the error as appropriate

#     try:
#         soup = BeautifulSoup(html_content.text, 'lxml')
#         container = soup.find('div', attrs={'data-state':'active'})
#         return container
#     except Exception as e:
#         print(f"Error during HTML parsing: {e}")
#         return None

In [3]:
def hashnode_data_points(html_body):
    all_articles = []

    for cont in html_body.find('div'):
        articles = cont.find('div', {'class':'flex flex-col gap-1 '})

    #extract article titles
        if cont.find('div', {'class':'flex flex-col gap-1'}) != None:
            article_titles = cont.find('h1').text

        #extract article authors name
        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-3'}) != None:
            names = cont.find('span', {'class':'font-semibold text-slate-700 dark:text-slate-200 cursor-pointer'}).text


        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-3'}) != None:
            dates = cont.find('p', {'class':'text-sm text-slate-500 dark:text-slate-400 font-normal'}).text


        #extracting the tag holding the article links
        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-1'}) != None:
            article_links = cont.find('div', {'class': 'flex flex-row items-center justify-start gap-1'}).find_all('a')
            if len(article_links) > 1:
                second_links = article_links[1]['href']


        #extracting the tag holding all likes and reads of the article
        if cont.find('section', {'class':'flex flex-col gap-5'}) != None:
            reactions = cont.find('section', {'class':'flex flex-col gap-5'}).find_all('p')
            if len(reactions) > 3:
                likes = reactions[1].text
                reads = reactions[3].text


        if cont.find('div', {'class':'hidden md:block'}) != None:
            article_contents = cont.find('span', {'class':'text-base hidden font-normal text-slate-500 dark:text-slate-400 hn-break-words cursor-pointer md:line-clamp-3'})

    #append all extracted tags object
        all_articles.append([article_titles, names, dates, second_links, likes, reads])
    return all_articles



In [4]:
hashnode_result = hashnode()

In [5]:
articles_dict_list=defaultdict(list)
articles_colnames = ['Title', 'AuthorName', 'DatePosted', 'ArticleLink', 'Likes', 'Reads']

In [6]:
articles_df = pd.DataFrame(hashnode_data_points(hashnode_result), columns=articles_colnames)

In [7]:
def article_content(links):
    request_link = requests.get(links)
    soup_content = BeautifulSoup(request_link.text, 'lxml')
    content = soup_content.find('div', {'id':'post-content-wrapper'}).text
    return content

articles_df['ArticleContent'] = articles_df.apply(lambda row: article_content(row['ArticleLink']), axis=1)

articles_df['ArticleContent'] = articles_df['ArticleContent'].replace(r'^\n', '', regex=True)
articles_df['ArticleContent'] = articles_df['ArticleContent'].replace(r'\n', ' ', regex=True)

In [8]:

def convert_to_date(value):
    try:
        return pd.to_datetime(value).date()
    except ValueError:
        return datetime.today().date()

articles_df['DatePosted'] = articles_df['DatePosted'].apply(convert_to_date)

In [9]:
articles_df.drop_duplicates(subset=['Title', 'AuthorName'], inplace=True)

In [10]:
articles_df

,Title,AuthorName,DatePosted,ArticleLink,Likes,Reads,ArticleContent
0,Deploying a Web Application with MongoDB on Ku...,Kyle Robins,2024-01-30,https://blog.kylerobins.com/deploying-a-web-ap...,32 likes,174 reads,Kubernetes has become the go-to solution for c...
1,Understanding Domain-Driven Design (Part 3),Ozan Akman,2024-01-29,https://compiler.blog/understanding-domain-dri...,39 likes,1.1K reads,Introduction Hello there! We’re back with our ...
3,Hashnode Academy: Your Go-To Tech Learning Hub,Victoria Lo,2024-02-03,https://lo-victoria.com/hashnode-academy,99 likes,71 reads,"Recently, the StackOverflow's 2023 Developer S..."
4,Introducing HashBlog - Generating a automated ...,Ishaan Dasgupta,2024-02-03,https://aquiem.hashnode.dev/introducing-hashbl...,41 likes,32 reads,"What is HashBlog ? To put it simply, HashBlog ..."
5,Introducing Hashyy - A companion for Online Co...,Sushil Bhattarai,2024-02-03,https://sushilbhattarai.hashnode.dev/introduci...,51 likes,40 reads,Hey! Have you ever found yourself immersed dee...
7,Hashtalk: Engage in Conversations with a Hashn...,Nocturnal,2024-02-03,https://aryan877.hashnode.dev/hashtalk-engage-...,22 likes,46 reads,Introduction: In the fast-changing world of te...
8,"Loid - Read, Write and Publish blogs on hashno...",Captain Jay Vijaykumar,2024-02-03,https://captainjay.hashnode.dev/loid-read-writ...,30 likes,33 reads,Ever been so busy programming that you just do...
10,Sentiment Analysis : Navigating Hashnode's API...,Shamitha Reddy Regenti,2024-02-03,https://shamithareddyregenti.hashnode.dev/sent...,30 likes,33 reads,The Initial Dilemma It all started with an ent...
11,Hashmind - Your hashnode companion,Benaiah,2024-02-03,https://benrobo.hashnode.dev/hashnode-blogging...,29 likes,58 reads,"To be honest, this article isn’t generated by ..."
12,Day 8 - Basic Git & GitHub,Aakash,2024-02-03,https://codewordsky.hashnode.dev/day-8-basic-g...,29 likes,58 reads,"Introduction In this blog, we are gonna have a..."


In [11]:
def clean_likes_reads(content):
    if "likes" in content:
        value = content.split()[0]
        if 'K' in value:
            return int(float(value[:-1]) * 1000)
        else:
            return int(value)
    elif "reads" in content:
        value = content.split()[0]
        if 'K' in value:
            return int(float(value[:-1]) * 1000)
        else:
            return int(value)
    else:
        return None


articles_df['Likes'] = articles_df['Likes'].apply(clean_likes_reads)
articles_df['Reads'] = articles_df['Reads'].apply(clean_likes_reads)

In [13]:
programming_languages = ['Python', 'Java', 'JavaScript', 'C', 'C++', 'C#', 'Ruby', 'PHP', 'Swift', 'Kotlin', \
                         'TypeScript','HTML', 'CSS', 'Go', 'Rust', 'MATLAB', 'Shell', 'PowerShell',\
                         'Scala', 'Perl', 'SQL']
articles_df['languages'] = articles_df['ArticleContent'].apply(lambda content: ', '.join([lang for lang in programming_languages if lang.lower() in content.lower()]))


In [20]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
def processing_content(content):
  analyzer = SentimentIntensityAnalyzer()
  sentiment_dict = analyzer.polarity_scores(content)
  if sentiment_dict['compound'] >= 0.05:
    return "Positve"
  elif sentiment_dict['compound'] <= - 0.05:
    return "Negative"
  else:
    return "Neutral"

In [24]:
articles_df['cleaned_content'] = articles_df['ArticleContent'].apply(processing_content)
articles_df.head()

,Title,AuthorName,DatePosted,ArticleLink,Likes,Reads,ArticleContent,languages,cleaned_content
0,Deploying a Web Application with MongoDB on Ku...,Kyle Robins,2024-01-30,https://blog.kylerobins.com/deploying-a-web-ap...,32,174,Kubernetes has become the go-to solution for c...,"C, Go",Positve
1,Understanding Domain-Driven Design (Part 3),Ozan Akman,2024-01-29,https://compiler.blog/understanding-domain-dri...,39,1100,Introduction Hello there! We’re back with our ...,"C, HTML, Go, Perl",Positve
3,Hashnode Academy: Your Go-To Tech Learning Hub,Victoria Lo,2024-02-03,https://lo-victoria.com/hashnode-academy,99,71,"Recently, the StackOverflow's 2023 Developer S...","Python, C, Go",Positve
4,Introducing HashBlog - Generating a automated ...,Ishaan Dasgupta,2024-02-03,https://aquiem.hashnode.dev/introducing-hashbl...,41,32,"What is HashBlog ? To put it simply, HashBlog ...","Java, JavaScript, C, Go",Positve
5,Introducing Hashyy - A companion for Online Co...,Sushil Bhattarai,2024-02-03,https://sushilbhattarai.hashnode.dev/introduci...,51,40,Hey! Have you ever found yourself immersed dee...,"Java, JavaScript, C, HTML, Go",Positve
